# Focus on One

In [ ]:
# ph_url = 'https://www.yelp.com/biz/joseph-peck-md-torrance'
ph_url = 'https://www.yelp.com/biz/chang-andrew-m-md-northridge'
# ph_url = 'https://www.yelp.com/biz/facial-aesthetic-concepts-san-clemente-2'
# ph_url = 'https://www.yelp.com/biz/michael-a-torres-md-baltimore'
ph_url

In [ ]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
import re
import requests
from scrapy.http import TextResponse
import json
from pprint import pprint

# we need headers to disguise our bot as a browser

headers = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
}



In [ ]:
url = ph_url
r = requests.get(url, headers = headers)
print(r.url)


# load the text to scrapy-type response
response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

# have a look at its body
print(str(response.body.decode()))

# with open('Yelp_Sample.html', 'w') as f:
#     f.write(str(response.body.decode()))


# Json Data

In [ ]:
xpath = './/script//text()'
selectors = response.xpath(xpath)
js_list = selectors.extract()
l = [i for i in js_list]
len(l)

## Json 14

In [ ]:
# idx = 14
json_list = [i for i in l if 'mainEntity' in i]
if len(json_list) == 1:
    json_string = json_list[0]
    # json_string = l[idx]
    d = json.loads(json_string)
else:
    d = {}
    
d

## Json 16 (Simple Reviews, drop) 

In [ ]:
# aim 2: get reviewCount
json_list = [i for i in l if 'LocalBusiness' in i and 'name' in i]
if len(json_list) == 1:
    json_string = json_list[0]
    d = json.loads(json_string)
    if 'aggregateRating' in d:
        reviewCount = d['aggregateRating']['reviewCount']
    else:
        reviewCount = 0
else:
    reviewCount = 0

reviewCount

## Json 19 (Basic Info)

In [ ]:
import html


# idx = 20

json_string = [i for i in l if 'ROOT_QUERY' in i][0]
x = json_string.replace('<!--', '').replace('-->', '')
# x = x.replace('&quot;', '"')
x = html.unescape(x)
# print(x)

x = json.loads(x)
# x = json.loads(x)


pprint(x)

In [ ]:
BusinessPhotos = [[i for i in x[i].values() if 'http' in i]  for i in x if '$BusinessPhoto' in i]
BusinessPhotos = [i[0] for i in BusinessPhotos  if len(i) == 1] 
BusinessPhotos

In [ ]:
# [i for i in x]

In [ ]:
root = x['ROOT_QUERY']


def decompose_jQuery(d, x):
    # d is input dict
    # x is the original list
    for k, v in d.items():
        if not v:
            # print('Empty v at {}'.format(k))
            pass
        
        elif type(v) == dict:
            print('Is a dictionary')
            # print(v)
            out = decompose_jQuery(v, x)
            if type(out) == dict: out = decompose_jQuery(out, x) 
            d[k] = out
        
        elif type(v) == list:
            if len(v) == sum([type(v_e) == dict for v_e in v]):
                print('Is a list of dictionaries')
                out_list = []
                for v_e in v:
                    out = decompose_jQuery(v_e, x) 
                    if type(out) == dict: out = decompose_jQuery(out, x) 
                    out_list.append(out)
                d[k] = out_list
            
        elif type(v) == str:
            # print('v is a string')
            if v[0] == '$' and v in x:
                # print(v)
                # if v == '$ROOT_QUERY.seoMetadata':
                #     print(v)
                print(v, 'v is start from $')
                d[k] = x[v]
            else:
                pass
        else:
            pass
            # print('Other Type')
            # print('Other Case v at {}'.format(k))
    return d

root = decompose_jQuery(root, x)


In [ ]:
[i for i in root]

root['seoMetadata']

In [ ]:
x['$ROOT_QUERY.seoMetadata']

In [ ]:
[# 'loggedInUser',
 # 'webConsumerFooter',
 # 'siteConfig',
 # 'webConsumerFooter({"baseUrl":"https://www.yelp.com/","urlOptions":{}})',
 # 'webHeader',
#  'mruLocation({"locationQueryParam":null})',
 'business({"encid":"vUSo8HMkVBiLwBGpFlMmSQ"})',
 # 'csrfToken({"scope":"/biz_info_api_v1/vUSo8HMkVBiLwBGpFlMmSQ/biz_pricing/update"})',
 # 'csrfToken({"scope":"/biz_info_api_v1/vUSo8HMkVBiLwBGpFlMmSQ/biz_pricing/delete"})',
 # 'csrfToken({"scope":"biz_photos_vote_form"})',
 # 'country({"code":"US"})',
 # 'csrfToken({"scope":"/signup/one_click_google"})',
#  'thirdPartyConfig',
 'seoMetadata'
]


business_col = [i for i in root if 'business' in i][0]
business_col


### Business

In [ ]:
buzid = root[business_col]['id']
# pprint(buz)
[i for i in buzid]


In [ ]:
['verifiedLicenses',
 # '__typename',
 'name',
 # 'categoryGroups',
 'alias',
 # 'externalResources',
 'menuVerbiage',
 # 'yelpMenu',
 'categories',
 # 'serviceUpdateSummary',
 'location',
 # 'serviceArea',
 # 'messaging',
 #'meteredPhoneNumber',
 'phoneNumber',
 # 'shouldHideContactInfoForMultilocationBusinessesExperiment',
 # 'jobPricing',
 'operationHours',
 'consumerAlert',
 # 'organizedProperties({"clientPlatform":"WWW"})',
 'healthInspections',
 'isCommunityQuestionsEnabled',
 'communityQuestions({"first":2})',
 'rating',
 'authoritativeAttributes',
 'containerBusiness',
 # 'map({"height":180,"width":315})',
 # 'map({"height":150,"width":315})',
 'serviceArea({"userType":"consumer"})',
 # 'media', 'buzPhoto'
 'reviewCount',
 'claimability({"useConsumerClaimability":true})',
 'claimability',
 'hasClaimReminderForCurrentUser',
 'alternateNames',
 'priceRange',
 'logo',
 'closedUntil',
 'primaryPhoto',
 'specialties',
 'history',
 # 'reviews({"first":20})'
]

pprint(buzid['history'])

In [ ]:
# pprint(buzid['media'])

In [ ]:
# pprint(buzid['categoryGroups'])

In [ ]:
pprint(buzid['categories'])

In [ ]:
pprint(buzid['operationHours'])

In [ ]:
# pprint(buzid['organizedProperties({"clientPlatform":"WWW"})'])

In [ ]:
pprint(buzid['authoritativeAttributes'])

In [ ]:
# buzid['reviews({"first":20})']

### SEO MetaData

In [ ]:
# [i for i in root['seoMetadata']]

# pprint(
seo_id = root['seoMetadata']['id']
[i for i in seo_id]
# seo_id['__typename']
seo_id

In [ ]:
bizDetail_col = [i for i in seo_id if 'bizDetails' in i][0]
bizDetails = seo_id[bizDetail_col]
# pprint(bizDetails)
[i for i in bizDetails]


In [ ]:
bizDetails_id = bizDetails['id']
# pprint(bizDetails_id)
[i for i in bizDetails_id]

In [ ]:
bizDetails_id['pageTitle']

In [ ]:
bizDetails_id['metaDescription']

In [ ]:
# bizDetails_id['canonicalUrl']

In [ ]:
# different language version; meaningless
# alternateTags = [x[i['id']] for i in bizDetails_id['alternateTags']]
# bizDetails_id['alternateTags']   

# Detailed Reviews

In [ ]:
encid = [i for i in root if 'business' in i][0]
encid = encid.split(':')[-1].split('"')[1]
encid

In [ ]:
L = []

for i in range(int(reviewCount/10) + 1):
    start = i * 10
    # print(start)
    url = 'https://www.yelp.com/biz/{}/review_feed?rl=en&q=&sort_by=date_desc&start={}'.format(encid, start)
    r = requests.get(url, headers = headers)
    print(r.url)
    # load the text to scrapy-type response
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

    # have a look at its body
    # print(str(response.body.decode()))

    # with open('Yelp_Sample.html', 'w') as f:
    #     f.write(str(response.body.decode()))

    reviews = response.json()
    # pprint(reviews)
    L.extend(reviews['reviews'])


In [ ]:
len(L)

# Not Recommended Reviews

In [ ]:
def process_reviewxpath(review):


    item2xpath = {
        'image': './/img[@class="photo-box-img"]/@src',
        'user_name': './/*[@class="user-display-name"]/text()',
        'user_location':'.//*[@class="user-location responsive-hidden-small"]/b/text()',
        'friends':'.//*[@class="friend-count responsive-small-display-inline-block"]//b/text()',
        'reviews':'.//*[@class="review-count responsive-small-display-inline-block"]//b/text()',
        'rate_score': './/img[@class="offscreen"]/@alt',
        'date': './/*[@class="rating-qualifier"]/text()',
        'comment': './/p[@lang="en"]//text()',
        # 'rate_details':'.//table[@class="rating-table"]//text()', 
    }



    itemselector2value = {
        'image': lambda x: x.extract()[0],
        'user_name': lambda x: x.extract()[0],
        'user_location': lambda x: x.extract()[0],
        'friends': lambda x: x.extract()[0],
        'reviews': lambda x: x.extract()[0],
        'rate_score': lambda x: x.extract()[0],
        'date': lambda x: x.extract()[0].strip(),
        'comment': lambda x: html.unescape('\n'.join(x.extract())).replace('\xa0',''),
        # 'rate_score': lambda x: float(x.extract()[0]), 
        # 'rate_num': lambda x: int(x.extract()[1]), 
        # 'rate_details': lambda x: x.extract(), 
    }


    d = {}
    for item, xpath in item2xpath.items():
        # print('\n')
        # print(item)
        selectors = review.xpath(xpath)#.extract()
        values = itemselector2value[item](selectors)
        d[item] = values
        # print(values)
        
        
    return d


def process_xpath(response):


    item2xpath = {
        'blocked_summary': './/*[@class="ysection not-recommended-reviews review-list-wide"]/h3/text()',
        'blocked_reviews': './/*[@class="ysection not-recommended-reviews review-list-wide"]//*[@class="review review--with-sidebar"]',
        
        'removed_summary': './/*[@class="ysection removed-reviews"]/h3/text()',
        'removed_reviews': './/*[@class="ysection removed-reviews"]//*[@class="review review--with-sidebar"]',
    }



    itemselector2value = {
        'blocked_summary': lambda x: int(x.extract()[0].strip().split(' ')[0]),
        'blocked_reviews': lambda x: [process_reviewxpath(review) for review in x],
        'removed_summary': lambda x: int(x.extract()[0].strip().split(' ')[0]) if len(x) == 1 else 0,
        'removed_reviews': lambda x: [process_reviewxpath(review) for review in x],
    }


    d = {}
    for item, xpath in item2xpath.items():
        # print('\n')
        # print(item)
        selectors = response.xpath(xpath)#.extract()
        values = itemselector2value[item](selectors)
        d[item] = values
        # print(values)
        
        
    return d

In [ ]:
def get_blocked_reviews_from_ph_url(ph_url):

    doc_name = ph_url.split('/')[-1]

    start = 0

    url = 'https://www.yelp.com/not_recommended_reviews/{}?not_recommended_start={}'.format(doc_name, start)
    r = requests.get(url, headers = headers)
    print(r.url)
    # load the text to scrapy-type response
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')


    d = process_xpath(response)
    blocked_reviews_num = int(d['blocked_summary'])
    if blocked_reviews_num <= 10:
        return d
    else:
        for i in range(1, int((blocked_reviews_num-1)/10) + 1):
            start = i*10
            print(start)
            url = 'https://www.yelp.com/not_recommended_reviews/{}?not_recommended_start={}'.format(doc_name, start)
            r = requests.get(url, headers = headers)
            print(r.url)
            # load the text to scrapy-type response
            response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
            new_d = process_xpath(response)
            d['blocked_reviews'] += new_d['blocked_reviews']
            # d['removed_reviews'] += new_d['removed_reviews']
            # print()
            # print(new_d['removed_reviews'])
        return d


print(ph_url)
d = get_blocked_reviews_from_ph_url(ph_url)

In [ ]:
len(d['blocked_reviews'])

In [ ]:
len(d['removed_reviews'])

# All In One

In [ ]:
import os
import requests
import pandas as pd
import time
import requests
from scrapy.http import TextResponse
import json
import html
import argparse
import random
from datetime import datetime



# we need headers to disguise our bot as a browser

headers = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
}



def decompose_jQuery(d, x):
    # d is input dict; x is the original list
    for k, v in d.items():
        if not v:
            # print('Empty v at {}'.format(k))
            pass
        
        elif type(v) == dict:
            # print('Is a dictionary')
            out = decompose_jQuery(v, x)
            if type(out) == dict: out = decompose_jQuery(out, x) 
            d[k] = out
        
        elif type(v) == list:
            if len(v) == sum([type(v_e) == dict for v_e in v]):
                # print('Is a list of dictionaries')
                out_list = []
                for v_e in v:
                    out = decompose_jQuery(v_e, x) 
                    if type(out) == dict: out = decompose_jQuery(out, x) 
                    out_list.append(out)
                d[k] = out_list
            
        elif type(v) == str:
            # print('v is a string')
            if v[0] == '$' and v in x:
                # print(v)
                # print(v, 'v is start from $')
                d[k] = x[v]
            else:
                pass
        else:
            pass
        
    return d


def process_reviewxpath(review):

    item2xpath = {
        'image': './/img[@class="photo-box-img"]/@src',
        'user_name': './/*[@class="user-display-name"]/text()',
        'user_location':'.//*[@class="user-location responsive-hidden-small"]/b/text()',
        'friends':'.//*[@class="friend-count responsive-small-display-inline-block"]//b/text()',
        'reviews':'.//*[@class="review-count responsive-small-display-inline-block"]//b/text()',
        'rate_score': './/img[@class="offscreen"]/@alt',
        'date': './/*[@class="rating-qualifier"]/text()',
        'comment': './/p[@lang="en"]//text()',
    }

    itemselector2value = {
        'image': lambda x: x.extract()[0],
        'user_name': lambda x: x.extract()[0],
        'user_location': lambda x: x.extract()[0],
        'friends': lambda x: x.extract()[0],
        'reviews': lambda x: x.extract()[0],
        'rate_score': lambda x: x.extract()[0],
        'date': lambda x: x.extract()[0].strip(),
        'comment': lambda x: html.unescape('\n'.join(x.extract())).replace('\xa0',''),
    }

    d = {}
    for item, xpath in item2xpath.items():
        # print('\n')
        # print(item)
        selectors = review.xpath(xpath)#.extract()
        values = itemselector2value[item](selectors)
        d[item] = values
        # print(values)
        
        
    return d


def process_xpath(response):
    item2xpath = {
        'blocked_reviews_num': './/*[@class="ysection not-recommended-reviews review-list-wide"]/h3/text()',
        'blocked_reviews': './/*[@class="ysection not-recommended-reviews review-list-wide"]//*[@class="review review--with-sidebar"]',
        'removed_reviews_num': './/*[@class="ysection removed-reviews"]/h3/text()',
        'removed_reviews': './/*[@class="ysection removed-reviews"]//*[@class="review review--with-sidebar"]',
    }


    itemselector2value = {
        'blocked_reviews_num': lambda x: int(x.extract()[0].strip().split(' ')[0]),
        'blocked_reviews': lambda x: [process_reviewxpath(review) for review in x],
        'removed_reviews_num': lambda x: int(x.extract()[0].strip().split(' ')[0]) if len(x) == 1 else 0,
        'removed_reviews': lambda x: [process_reviewxpath(review) for review in x],
    }


    d = {}
    for item, xpath in item2xpath.items():
        # print('\n')
        # print(item)
        selectors = response.xpath(xpath)#.extract()
        values = itemselector2value[item](selectors)
        d[item] = values
        # print(values)
    return d


def get_blocked_reviews_from_ph_url(ph_url):
    doc_name = ph_url.split('/')[-1]
    start = 0
    url = 'https://www.yelp.com/not_recommended_reviews/{}?not_recommended_start={}'.format(doc_name, start)
    r = requests.get(url, headers = headers)
    print(r.url)
    # load the text to scrapy-type response
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')


    d = process_xpath(response)
    blocked_reviews_num = int(d['blocked_reviews_num'])
    if blocked_reviews_num <= 10:
        return d
    else:
        for i in range(1, int((blocked_reviews_num)/10) + 1):
            start = i*10
            # print(start)
            url = 'https://www.yelp.com/not_recommended_reviews/{}?not_recommended_start={}'.format(doc_name, start)
            r = requests.get(url, headers = headers)
            print(r.url)
            # load the text to scrapy-type response
            response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
            new_d = process_xpath(response)
            d['blocked_reviews'] += new_d['blocked_reviews']
            # d['removed_reviews'] += new_d['removed_reviews']
            # print()
            # print(new_d['removed_reviews'])
        return d


def get_physician_info_from_yelp_url(ph_url):

    physician_info = {}
    
    # profile url
    url = ph_url
    r = requests.get(url, headers = headers)
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
    xpath = './/script//text()'
    selectors = response.xpath(xpath)
    js_list = selectors.extract()
    l = [i for i in js_list]

    # aim 1: get common questions
    # idx = 14
    json_list = [i for i in l if 'mainEntity' in i]
    if len(json_list) == 1:
        json_string = json_list[0]
        d = json.loads(json_string)
    else:
        d = {}
    physician_info['Questions'] = d

    
    # aim 2: get reviewCount
    json_list = [i for i in l if 'LocalBusiness' in i and 'name' in i]
    if len(json_list) == 1:
        json_string = json_list[0]
        d = json.loads(json_string)
        if 'aggregateRating' in d:
            reviewCount = d['aggregateRating']['reviewCount']
        else:
            reviewCount = 0
    else:
        reviewCount = 0
    physician_info['reviewCount'] = reviewCount


    # aim 3: get physician basic information
    # idx = 19
    # json_string = l[idx]
    json_string = [i for i in l if 'ROOT_QUERY' in i][0]
    x = json_string.replace('<!--', '').replace('-->', '')
    # x = x.replace('&quot;', '"')
    x = html.unescape(x)
    # print(x)
    x = json.loads(x)

    # aim 3.1: business photos
    BusinessPhotos = [[i for i in x[i].values() if 'http' in i]  for i in x if '$BusinessPhoto' in i]
    BusinessPhotos = [i[0] for i in BusinessPhotos  if len(i) == 1] 
    physician_info['BusinessPhotos'] = BusinessPhotos

    # aim 3.2: get buzid info
    root = x['ROOT_QUERY']
    root = decompose_jQuery(root, x)
    business_col = [i for i in root if 'business' in i][0]
    buzid = root[business_col]['id']

    cols = ['verifiedLicenses',
    # '__typename',
    'name',
    # 'categoryGroups',
    'alias',
    # 'externalResources',
    'menuVerbiage',
    # 'yelpMenu',
    'categories',
    # 'serviceUpdateSummary',
    'location',
    # 'serviceArea',
    # 'messaging',
    #'meteredPhoneNumber',
    'phoneNumber',
    # 'shouldHideContactInfoForMultilocationBusinessesExperiment',
    # 'jobPricing',
    'operationHours',
    'consumerAlert',
    # 'organizedProperties({"clientPlatform":"WWW"})',
    'healthInspections',
    'isCommunityQuestionsEnabled',
    'communityQuestions({"first":2})',
    'rating',
    'authoritativeAttributes',
    'containerBusiness',
    # 'map({"height":180,"width":315})',
    # 'map({"height":150,"width":315})',
    'serviceArea({"userType":"consumer"})',
    # 'media', 'buzPhoto'
    'reviewCount',
    'claimability({"useConsumerClaimability":true})',
    'claimability',
    'hasClaimReminderForCurrentUser',
    'alternateNames',
    'priceRange',
    'logo',
    'closedUntil',
    'primaryPhoto',
    'specialties',
    'history',]

    for col in cols:
        physician_info[col] = buzid[col]

    # aim 3.3: get seo info 
    seo_id = root['seoMetadata']['id']
    # [i for i in seo_id]
    # seo_id['__typename']
    bizDetail_col = [i for i in seo_id if 'bizDetails' in i][0]
    bizDetails = seo_id[bizDetail_col]

    bizDetails_id = bizDetails['id']

    cols_second = ['pageTitle', 'metaDescription' ]
    for col in cols_second:
        physician_info[col] = bizDetails_id[col]

    # aim 4: get encid and good reviews
    encid = [i for i in root if 'business' in i][0]
    encid = encid.split(':')[-1].split('"')[1]
    physician_info['encid'] = encid
    
    L = []
    for i in range(int((reviewCount)/10) + 1):
        start = i * 10
        url = 'https://www.yelp.com/biz/{}/review_feed?rl=en&q=&sort_by=date_desc&start={}'.format(encid, start)
        r = requests.get(url, headers = headers)
        print(r.url)
        # load the text to scrapy-type response
        response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

        reviews = response.json()
        L.extend(reviews['reviews'])

    physician_info['reviews_detailed'] = L

    # aim 5: get blocked reivews
    d = get_blocked_reviews_from_ph_url(ph_url)
    for k, v in d.items(): physician_info[k] = v
    
    return physician_info


In [ ]:
# ph_url = 'https://www.yelp.com/biz/joseph-peck-md-torrance' # many reviews 
# ph_url = 'https://www.yelp.com/biz/integrated-pain-management-tarzana-6' # a few reviews
# ph_url = 'https://www.yelp.com/biz/michael-shabtai-md-northridge?page_src=related_bizes' # even fewer reviews

ph_info = get_physician_info_from_yelp_url(ph_url)

In [ ]:
pprint(ph_info)

In [ ]:

print('Good    Reviews: reported {} vs collected {}'.format(ph_info['reviewCount'], len(ph_info['reviews_detailed'])))
print('Blocked Reviews: reported {} vs collected {}'.format(ph_info['blocked_reviews_num'], len(ph_info['blocked_reviews'])))
print('Removed Reviews: reported {} vs collected {}'.format(ph_info['removed_reviews_num'], len(ph_info['removed_reviews'])))
